# Rescheduling mit Flow Time
> Planunghorizont Tag 0: 2 Tage
> Rescheduling des Tages 1

In [ ]:
from configs.config import get_path
import src.utils.converter as convert
import src.utils.presenter as show
import src.utils.checker as check

import src.utils.gen_interarrival as gen_interarrival
import src.utils.gen_jobs as gen_jobs

import time
import pandas as pd
pd.set_option('display.max_rows', 26)

In [ ]:
import src.models.lp.flowtime_solver as solver
from src.simulation.ProductionDaySimulation import ProductionDaySimulation

In [ ]:
max_time = 60*60*1  # 1h

In [ ]:
#import utils.scheduling_solver as ssv
#import utils.rescheduling_solver as rssv
#from ProductionDaySimulation import ProductionDaySimulation

In [ ]:
basic_data_path = get_path("data", "basic")

df_instance = pd.read_csv(basic_data_path / "00_instance.csv")
df_instance

### 1) Aufträge generieren

In [ ]:
df_jssp = gen_jobs.create_multiple_jobs(df_instance, repetitions = 4, shuffle = True)
df_jssp

### 2) Mittlere Interarrival-Zeit berechnen

In [ ]:
t_a = gen_interarrival.calculate_mean_interarrival_time(df_jssp, u_b_mmax= 0.92)
t_a

### 3) Ankunftszeiten erzeugen

In [ ]:
df_jobs_arrivals = gen_interarrival.generate_arrivals(df_jssp, mean_interarrival_time=t_a, start_time=0.0)
df_jobs_arrivals

### 4) Nur Jobs behalten, deren Ankunft im Zeitfenster liegt

In [ ]:
day_count = 1

time_limit = 60 * 24 * day_count
df_jobs_arrivals = df_jobs_arrivals[df_jobs_arrivals['Arrival'] < time_limit].reset_index(drop=True)
df_jobs_arrivals

In [ ]:
valid_jobs = set(df_jobs_arrivals['Job'])
df_jssp = df_jssp[df_jssp['Job'].isin(valid_jobs)].reset_index(drop=True)
df_jssp

In [ ]:
# df_jssp, df_jobs_arrivals = gen_jobs.create_jobs_for_days(df_instance, day_count = 1, u_b_mmax= 0.92, shuffle = True)
# df_jobs_arrivals

## I) Complete Scheduling

In [ ]:
starting_time = time.time()

# Scheduling
df_plan = solver.solve_jssp_flowtime(df_jssp, df_jobs_arrivals, solver= "HiGHS", msg=False, timeLimit=max_time, gapRel= 0.10)
df_plan

# Informationen
ending_time = time.time()
solver_duration = ending_time - starting_time
print(f"\nScheduling-Dauer: {int(solver_duration // 60)} Minuten und {(solver_duration % 60):.2f} Sekunden.")
df_plan

In [ ]:
show.plot_gantt_machines(df_plan)

check.is_machine_conflict_free(df_plan)
check.is_operation_sequence_correct(df_plan)
check.is_job_timing_correct(df_plan)
check.is_start_correct(df_plan)

## II) Erster Tag (Tag 0)

### a) Simulation

In [ ]:
day_length = 1440

In [ ]:
day_numb = 0

day_start = day_length*day_numb 
day_end = day_start + day_length

In [ ]:
simulation = ProductionDaySimulation(df_plan, vc=0.25)
df_execution, df_undone = simulation.run(start_time = day_start, end_time=day_end)

In [ ]:
df_undone

In [ ]:
if not df_execution.empty:
    show.plot_gantt_machines(df_execution, title=f"Gantt-Diagramm ab Tag {day_numb}", duration_column="Simulated Processing Time")
else:
    print(f"Nothing executed on day {day_numb}")

### b) Verbleibene Jobs (Rest)

In [ ]:
df_rest = df_plan.merge(df_execution[["Job", "Machine"]], on=["Job", "Machine"], how="left", indicator=True)
df_rest = df_rest[df_rest["_merge"] == "left_only"].drop(columns=["_merge"])
df_rest = df_rest.sort_values(by=["Arrival", "Start"]).reset_index(drop=True)
df_rest

In [ ]:
allowed = df_rest["Job"].unique() 
df_time_rest = df_jobs_arrivals[df_jobs_arrivals["Job"].isin(allowed)].copy()
df_time_rest

## III) Rescheduling
> vereinfacht, ohne neu Aufträge (nur der Rest)

In [ ]:
# Reduktion der bereits ausgeführten Operationen auf die relevanten für die nächsten Tage
df_execution_important = df_execution[df_execution["End"] >= 1440]
df_execution_important

In [ ]:
df_reschedule = solver.solve_jssp_individual_flowtime_with_fixed_ops(df_rest, df_time_rest, df_execution_important,
                                                                   reschedule_start = 1440,
                                                                   solver= "HiGHS", msg=False, timeLimit=max_time, gapRel= 0.10)
df_reschedule

In [ ]:
show.plot_gantt_machines(df_reschedule)
check.is_machine_conflict_free(df_reschedule)
check.is_operation_sequence_correct(df_reschedule)
check.is_job_timing_correct(df_reschedule)
check.is_start_correct(df_reschedule)